In [5]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def deriv_sigmoid(x):
    fx = sigmoid(x)
    return fx * (1 - fx)


def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()


class OurNeuralNetwork:
    def __init__(self):
        self.weights = [np.random.normal() for _ in range(6)]
        self.biases = [np.random.normal() for _ in range(3)]

    def feedforward(self, x):
        h1_input = np.dot(x, self.weights[:2]) + self.biases[0]
        h1 = sigmoid(h1_input)
        h2_input = np.dot(x, self.weights[2:4]) + self.biases[1]
        h2 = sigmoid(h2_input)
        o1_input = np.dot([h1, h2], self.weights[4:]) + self.biases[2]
        return sigmoid(o1_input)

    def train(self, data, all_y_trues, learn_rate=0.1, epochs=1000):
        for epoch in range(epochs):
            for x, y_true in zip(data, all_y_trues):
                h1_input = np.dot(x, self.weights[:2]) + self.biases[0]
                h1 = sigmoid(h1_input)
                h2_input = np.dot(x, self.weights[2:4]) + self.biases[1]
                h2 = sigmoid(h2_input)
                o1_input = np.dot([h1, h2], self.weights[4:]) + self.biases[2]
                o1 = sigmoid(o1_input)

                d_L_d_ypred = -2 * (y_true - o1)
                d_ypred_d_w5 = h1 * deriv_sigmoid(o1_input)
                d_ypred_d_w6 = h2 * deriv_sigmoid(o1_input)
                d_ypred_d_b3 = deriv_sigmoid(o1_input)
                d_ypred_d_h1 = self.weights[4] * deriv_sigmoid(o1_input)
                d_ypred_d_h2 = self.weights[5] * deriv_sigmoid(o1_input)

                d_h1_d_w1 = x[0] * deriv_sigmoid(h1_input)
                d_h1_d_w2 = x[1] * deriv_sigmoid(h1_input)
                d_h2_d_w3 = x[0] * deriv_sigmoid(h2_input)
                d_h2_d_w4 = x[1] * deriv_sigmoid(h2_input)

                self.weights[0] -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
                self.weights[1] -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
                self.biases[0] -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * deriv_sigmoid(h1_input)

                self.weights[2] -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
                self.weights[3] -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
                self.biases[1] -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * deriv_sigmoid(h2_input)

                self.weights[4] -= learn_rate * d_L_d_ypred * d_ypred_d_w5
                self.weights[5] -= learn_rate * d_L_d_ypred * d_ypred_d_w6
                self.biases[2] -= learn_rate * d_L_d_ypred * d_ypred_d_b3

            if epoch % 100 == 0:
                y_preds = np.apply_along_axis(self.feedforward, 1, data)
                loss = mse_loss(all_y_trues, y_preds)
                print(f"Epoch {epoch} loss: {loss:.3f}")

    def predict(self, data):
        predictions = [self.feedforward(x) for x in data]
        return np.round(predictions)


# Определим набор данных
data = np.array(
    [
        [-2, -1],  # Алиса
        [25, 6],  # Боб
        [17, 4],  # Чарли
        [-15, -6],  # Диана
    ]
)
all_y_trues = np.array(
    [
        1,  # Алиса
        0,  # Боб
        0,  # Чарли
        1,  # Диана
    ]
)

# Создание экземпляра и обучение собственной нейронной сети
network = OurNeuralNetwork()
network.train(data, all_y_trues)

# Оценка точности собственной нейронной сети
predictions = network.predict(data)
accuracy = np.mean(predictions == all_y_trues)
print(f"NeuralNetwork Accuracy: {accuracy * 100:.2f}%")

# Создание и обучение MLPClassifier от sklearn
mlp = MLPClassifier(hidden_layer_sizes=(2,), activation="logistic", max_iter=2000, random_state=42)
mlp.fit(data, all_y_trues)

# Оценка точности MLPClassifier от sklearn
mlp_accuracy = mlp.score(data, all_y_trues)
print(f"MLPClassifier Accuracy: {mlp_accuracy * 100:.2f}%")

# Данные для прогнозирования
new_data = np.array([[-10, 5], [15, 3]])  # Новый пример 1  # Новый пример 2

# Прогноз собственной нейронной сети
network_predictions = network.predict(new_data)
print("NeuralNetwork Predictions:")
for i, pred in enumerate(network_predictions):
    print(f"Example {i+1}: {'woman' if pred == 1 else 'man'}")

# Прогноз MLPClassifier от sklearn
mlp_predictions = mlp.predict

Epoch 0 loss: 0.288
Epoch 100 loss: 0.021
Epoch 200 loss: 0.009
Epoch 300 loss: 0.006
Epoch 400 loss: 0.004
Epoch 500 loss: 0.003
Epoch 600 loss: 0.003
Epoch 700 loss: 0.002
Epoch 800 loss: 0.002
Epoch 900 loss: 0.002
NeuralNetwork Accuracy: 100.00%
MLPClassifier Accuracy: 100.00%
NeuralNetwork Predictions:
Example 1: woman
Example 2: man
